<a href="https://colab.research.google.com/github/simonxmachine/news_sentiment_analysis/blob/main/news_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests

plt.style.use('ggplot')

import nltk

In [ ]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('vader_lexicon')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
#Fetch data from NewsApi.org

api_key = '6240a9d79cfa443cad793c2d0468c36e'
search_topic = 'solana'
url = f"https://newsapi.org/v2/everything?q={search_topic}&apiKey={api_key}"

# Send a GET request to the API endpoint
response = requests.get(url)

# Check for successful response
if response.status_code == 200:
    # Parse the JSON data
    data = response.json()

else:
    print("Error:", response.status_code)


In [ ]:
keys = [key for key in data.keys()]

data['articles'][3]

{'source': {'id': 'business-insider', 'name': 'Business Insider'},
 'author': 'Katherine Tangalakis-Lippert,Lloyd Lee',
 'title': 'Former Amazon engineer pleads guilty to stealing $12.3 million of crypto in first ever hacking case involving smart contracts',
 'description': 'A former Amazon engineer pleaded guilty to hacking two crypto exchanges and stealing $12.3 million in the first ever conviction involving smart contracts.',
 'url': 'https://www.businessinsider.com/amazon-engineer-hacking-cryptocurrency-nirvana-solana-what-are-smart-contracts-2023-12',
 'urlToImage': 'https://i.insider.com/657e49ed50edbc52a865111a?width=1200&format=jpeg',
 'publishedAt': '2023-12-17T04:53:16Z',
 'content': 'Representation of Bitcoin is seen with binary code displayed on a laptop screenNurPhoto/Getty Images\r\n<ul><li>An ex-Amazon engineer pleaded guilty this week to hacking two cryptocurrency exchanges.</… [+3918 chars]'}

In [ ]:
for i in range(10):

  print(data['articles'][i]['content'])

Open-source blockchain platform, Solana’s native cryptocurrency, SOL, neared the $100 mark on Friday morning, extending a powerful rally into its third consecutive week.
The price surge has coincide… [+2811 chars]
Former Amazon security engineer Shakeeb Ahmed pleaded guilty this week to hacking and stealing over $12.3 million from two cryptocurrency exchanges in July 2022. The two affected companies are Nirvan… [+2068 chars]
Shakeeb Ahmed, a former security engineer at Amazon, has pleaded guilty to charges of hacking and stealing over $12.3 million from two crypto exchanges in July 2022. This high-profile cybercrime inci… [+3117 chars]
Representation of Bitcoin is seen with binary code displayed on a laptop screenNurPhoto/Getty Images
<ul><li>An ex-Amazon engineer pleaded guilty this week to hacking two cryptocurrency exchanges.</… [+3918 chars]
Im Juni 2022 vollmundig angekündigt, hat sich das auf Web3-Anwendungen spezialisierte Krypto-Smartphone Solana Saga seit dem Erscheinen eher z

In [ ]:
len(data['articles'])

100

In [ ]:
#Produce the dataframe for the articles

df = pd.DataFrame([dict(article) for article in data["articles"]], columns=["source", "author", "title", "description", "url", "urlToImage", "publishedAt", "content"])

#Add ID column
df['Id'] = range(1, len(df) + 1)

#Rearrange the rows
df = df[['Id', 'source', 'author', 'title', 'description', 'url', 'urlToImage', 'publishedAt', 'content']]


df.head(3)

,Id,source,author,title,description,url,urlToImage,publishedAt,content
0,1,"{'id': None, 'name': 'ReadWrite'}",Sam Shedden,Solana price nears $100 amid meme coin surge,"Open-source blockchain platform, Solana’s nati...",https://readwrite.com/solana-price-nears-100-a...,https://readwrite.com/wp-content/uploads/2023/...,2023-12-22T15:33:04Z,"Open-source blockchain platform, Solana’s nati..."
1,2,"{'id': None, 'name': 'Slashdot.org'}",BeauHD,Ex-Amazon Engineer Pleads Guilty To Hacking Cr...,An anonymous reader quotes a report from Bleep...,https://slashdot.org/story/23/12/19/0730258/ex...,https://a.fsdn.com/sd/topics/bitcoin_64.png,2023-12-19T13:00:00Z,Former Amazon security engineer Shakeeb Ahmed ...
2,3,"{'id': None, 'name': 'ReadWrite'}",Maxwell William,Ex-Amazon security engineer admits to stealing...,"Shakeeb Ahmed, a former security engineer at A...",https://readwrite.com/ex-amazon-security-engin...,https://readwrite.com/wp-content/uploads/2023/...,2023-12-17T02:52:53Z,"Shakeeb Ahmed, a former security engineer at A..."


In [ ]:
df['description']

0     Open-source blockchain platform, Solana’s nati...
1     An anonymous reader quotes a report from Bleep...
2     Shakeeb Ahmed, a former security engineer at A...
3     A former Amazon engineer pleaded guilty to hac...
4     Weil Besitzer:innen des Krypto-Smartphones Sol...
                            ...                        
95    What to watch for in 2024, according to market...
96    The Solana ecosystem has been a worthy contend...
97    La entidad medió en la negociación del fin de ...
98    The total value locked (TVL) of all Cardano-ba...
99    Crypto investment funds last week attracted th...
Name: description, Length: 100, dtype: object

# Basic NLTK


In [ ]:
#Natural Language Tool Kit

example = df['content'][0]
print(example)

Open-source blockchain platform, Solana’s native cryptocurrency, SOL, neared the $100 mark on Friday morning, extending a powerful rally into its third consecutive week.
The price surge has coincide… [+2811 chars]


In [ ]:
# Tokenizing - destructure all the words up into individual strings in a list as tokens

tokens = nltk.word_tokenize(example)
tokens[:10]

['Open-source',
 'blockchain',
 'platform',
 ',',
 'Solana',
 '’',
 's',
 'native',
 'cryptocurrency',
 ',']

In [ ]:
#Find part of speech tags

tagged = nltk.pos_tag(tokens)
tagged [:10]

[('Open-source', 'NNP'),
 ('blockchain', 'NN'),
 ('platform', 'NN'),
 (',', ','),
 ('Solana', 'NNP'),
 ('’', 'NNP'),
 ('s', 'VBD'),
 ('native', 'JJ'),
 ('cryptocurrency', 'NN'),
 (',', ',')]

In [ ]:
#Break the words into chunks with attempts to classify some of the tokens

entities = nltk.chunk.ne_chunk(tagged)
#entities.pprint()

# VADER SENTIMENT SCORING

### Uses NLTK's SentimentIntensityAnalyzer to get the neg/neu/pos scores of the individual text

#### 1. Stop words are removed
#### 2. Each word is scored and combined to a total score

In [ ]:
#VADER (Valence Aware Dictionary and sEntiment Reasoner) - Bag of words approach

from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm

#Instantiate the object
sia = SentimentIntensityAnalyzer()

In [ ]:
#Analyze how negative or positive the string is.
#The compound score takes the 3 scores and provides a value between -1 to 1 to
#assess how positive or negative it is

sia.polarity_scores('I AM VERY UPSET')

{'neg': 0.692, 'neu': 0.308, 'pos': 0.0, 'compound': -0.6697}

In [ ]:
sia.polarity_scores(example)

{'neg': 0.0, 'neu': 0.909, 'pos': 0.091, 'compound': 0.4215}

In [ ]:
# Run the polarity score on the entire dataset

res = {}

for i, row in tqdm(df.iterrows(), total=len(df)):
  text = row['content']
  myid = row['Id']
  res[myid] = sia.polarity_scores(text)

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
res[5]


{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

In [ ]:
#Turn the result into a dataframe and Transpose to flip chart proper way

vaders = pd.DataFrame(res).T

#Reset the index on vaders DF to make more manageable, and rename to Id
vaders = vaders.reset_index().rename(columns={'index': 'Id'})


In [ ]:
#Merge the vaders DF to the original DF

vaders = vaders.merge(df, how='left')

In [ ]:
vaders.head(2)

,Id,neg,neu,pos,compound,source,author,title,description,url,urlToImage,publishedAt,content
0,1,0.000,0.909,0.091,0.4215,"{'id': None, 'name': 'ReadWrite'}",Sam Shedden,Solana price nears $100 amid meme coin surge,"Open-source blockchain platform, Solana’s nati...",https://readwrite.com/solana-price-nears-100-a...,https://readwrite.com/wp-content/uploads/2023/...,2023-12-22T15:33:04Z,"Open-source blockchain platform, Solana’s nati..."
1,2,0.207,0.689,0.105,-0.6124,"{'id': None, 'name': 'Slashdot.org'}",BeauHD,Ex-Amazon Engineer Pleads Guilty To Hacking Cr...,An anonymous reader quotes a report from Bleep...,https://slashdot.org/story/23/12/19/0730258/ex...,https://a.fsdn.com/sd/topics/bitcoin_64.png,2023-12-19T13:00:00Z,Former Amazon security engineer Shakeeb Ahmed ...


In [ ]:
average = vaders['compound'].mean()

In [ ]:
print(average)

0.203324


# Using the Roberta Pretrained Model from Hugging Face


*   Use a model trained on a large corpus of data
*   Transformer model accounts for the words and the contex in relation to the other words




In [ ]:
from transformers import AutoTokenizer #similar to what NLTK did
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

In [ ]:
#Using a trained model with weights from Hugging Face pre-trained model on Twitter labels
#Leverage the power of pre-trained language models, which have already learned patterns from massive
#amounts of text data. This saves time and effort in training a model from scratch.

MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [ ]:
example = "In New York City, John Wick prepares to exact vengeance against the High Table while hiding underground with the Bowery King. He travels to Morocco and kills the Elder, the 'one who sits above the Table'. In response, High Table member Marquis Vincent Bisset de Gramont summons New York Continental hotel manager Winston and his concierge Charon, where he explains that the High Table has given him unlimited resources to kill John and chastises Winston for previously failing to do so. As punishment, the Marquis strips Winston of his managerial duties, declares him 'excommunicado, destroys the New York Continental and executes Charon. The Marquis then enlists Caine – a blind, retired High Table assassin – to kill his old friend John, threatening to murder Caine's daughter. John takes refuge at the Osaka Continental, run by his friend Shimazu Koji. The Marquis' second-in-command Chidi, backed by High Table assassins and Caine, arrives to investigate the hotel. Koji's daughter Akira, the hotel's concierge, evacuates the hotel just before the High Table 'deconsecrates' it, igniting a melee. John fights through waves of armored assassins, leading to a showdown against Caine. A bounty hunter named Mr. Nobody interrupts their fight and allows John to escape after determining the current bounty for killing John to be insufficient. A wounded Koji repeatedly attacks Caine on John's behalf, until Caine reluctantly kills Koji, but spares Akira. John returns to New York and meets with a vengeful Winston at Charon's gravesite. Winston suggests that John invoke an old High Table tradition to challenge the Marquis to a duel. Winning would free John of all obligations to the High Table, but he can only request a duel on behalf of a crime family. John travels to the Berlin headquarters of the Ruska Roma crime syndicate, with whom he had severed ties, to request readmission. His adoptive sister Katia stipulates that for readmission, John must kill Killa Harkan, a High Table member who murdered her father. Killa sets up an ambush at his nightclub, but with the aid of Caine and Mr. Nobody, John manages to kill him and regain his status. "

In [ ]:
# Encode the sample text by tokenizing and returning as Pytorch Tensor
# Then run the encoded data with the pre-trained model
# Then use softmax to normalize the data into probabilities that sum up to 1

encoded_text = tokenizer(example, return_tensors='pt')
output = model(**encoded_text)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
scores_dict = {
    'roberta_neg' : scores[0],
    'roberta_neu' : scores[1],
    'roberta_pos' : scores[2]
}

print(scores_dict)

{'roberta_neg': 0.3595628, 'roberta_neu': 0.61708313, 'roberta_pos': 0.023354072}


In [ ]:
scores

array([0.3595628 , 0.61708313, 0.02335407], dtype=float32)

In [ ]:
# Define the function to take the input data to find Roberta polarity scores

def polarity_scores_roberta(sample_text):
  encoded_text = tokenizer(example, return_tensors='pt')
  output = model(**encoded_text)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)
  scores_dict = {
      'roberta_neg' : scores[0],
      'roberta_neu' : scores[1],
      'roberta_pos' : scores[2]
  }

  return scores_dict

In [ ]:
res = {}

for i, row in tqdm(df.iterrows(), total=len(df)):
  try:
    text = row['content']
    myid = row['Id']
    vader_result = sia.polarity_scores(text)
    vader_result_rename = {}
    for key, value in vader_result.items():
      vader_result_rename[f"vader_{key}"] = value

    roberta_result = polarity_scores_roberta(text)
    both = {**vader_result, **roberta_result}
    res[myid] = both
  except RuntimeError:
    print(f'Broke for id {myid}')

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
#Turn the result into a dataframe and Transpose to flip chart proper way

results_df = pd.DataFrame(res).T

#Reset the index on vaders DF to make more manageable, and rename to Id
results_df = results_df.reset_index().rename(columns={'index': 'Id'})

results_df = results_df.merge(df, how='left')


In [ ]:
results_df

,Id,neg,neu,pos,compound,roberta_neg,roberta_neu,roberta_pos,source,author,title,description,url,urlToImage,publishedAt,content
0,1,0.000,0.909,0.091,0.4215,0.359563,0.617083,0.023354,"{'id': None, 'name': 'ReadWrite'}",Sam Shedden,Solana price nears $100 amid meme coin surge,"Open-source blockchain platform, Solana’s nati...",https://readwrite.com/solana-price-nears-100-a...,https://readwrite.com/wp-content/uploads/2023/...,2023-12-22T15:33:04Z,"Open-source blockchain platform, Solana’s nati..."
1,2,0.207,0.689,0.105,-0.6124,0.359563,0.617083,0.023354,"{'id': None, 'name': 'Slashdot.org'}",BeauHD,Ex-Amazon Engineer Pleads Guilty To Hacking Cr...,An anonymous reader quotes a report from Bleep...,https://slashdot.org/story/23/12/19/0730258/ex...,https://a.fsdn.com/sd/topics/bitcoin_64.png,2023-12-19T13:00:00Z,Former Amazon security engineer Shakeeb Ahmed ...
2,3,0.217,0.680,0.103,-0.6705,0.359563,0.617083,0.023354,"{'id': None, 'name': 'ReadWrite'}",Maxwell William,Ex-Amazon security engineer admits to stealing...,"Shakeeb Ahmed, a former security engineer at A...",https://readwrite.com/ex-amazon-security-engin...,https://readwrite.com/wp-content/uploads/2023/...,2023-12-17T02:52:53Z,"Shakeeb Ahmed, a former security engineer at A..."
3,4,0.097,0.903,0.000,-0.4215,0.359563,0.617083,0.023354,"{'id': 'business-insider', 'name': 'Business I...","Katherine Tangalakis-Lippert,Lloyd Lee",Former Amazon engineer pleads guilty to steali...,A former Amazon engineer pleaded guilty to hac...,https://www.businessinsider.com/amazon-enginee...,https://i.insider.com/657e49ed50edbc52a865111a...,2023-12-17T04:53:16Z,Representation of Bitcoin is seen with binary ...
4,5,0.000,1.000,0.000,0.0000,0.359563,0.617083,0.023354,"{'id': 't3n', 'name': 'T3n'}",Jörn Brien,Bonk-Airdrop sorgt für Ausverkauf bei Krypto-S...,Weil Besitzer:innen des Krypto-Smartphones Sol...,https://t3n.de/news/bonk-airdrop-smartphone-so...,https://t3n.de/news/wp-content/uploads/2023/12...,2023-12-17T20:51:55Z,"Im Juni 2022 vollmundig angekündigt, hat sich ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,0.000,0.867,0.133,0.5994,0.359563,0.617083,0.023354,"{'id': None, 'name': 'CoinDesk'}",Lyllah Ledesma,"Solana, Avax, Helium Led Digital Assets Gains ...","What to watch for in 2024, according to market...",https://www.coindesk.com/markets/2023/12/21/so...,https://www.coindesk.com/resizer/yob9dsyb8vwt9...,2023-12-21T10:41:23Z,Layer-1 blockchain Solana (SOL) led the way in...
96,97,0.000,0.808,0.192,0.7096,0.359563,0.617083,0.023354,"{'id': None, 'name': 'Techreport.com'}",Nick Dunn,Solana Price Prediction: SOL Breaches $67 Leve...,The Solana ecosystem has been a worthy contend...,https://techreport.com/price-prediction/solana...,https://techreport.com/wp-content/uploads/2023...,2023-12-09T05:59:58Z,The Solana ecosystem has been a worthy contend...
97,98,0.000,1.000,0.000,0.0000,0.359563,0.617083,0.023354,"{'id': None, 'name': '20minutos.es'}",Miriam Cos Manuel,"La fundación Henry Dunant, mediador en el desa...",La entidad medió en la negociación del fin de ...,https://www.20minutos.es/noticia/5195336/0/fun...,https://imagenes.20minutos.es/files/image_1920...,2023-11-30T21:54:44Z,El Centro para el Diálogo Humanitario Henry Du...
98,99,0.000,0.863,0.137,0.6249,0.359563,0.617083,0.023354,"{'id': None, 'name': 'CoinDesk'}",Shaurya Malwa,Total Value of Cardano DeFi Ecosystem Nears $4...,The total value locked (TVL) of all Cardano-ba...,https://www.coindesk.com/markets/2023/12/14/to...,https://www.coindesk.com/resizer/NSFaR1EOEVwsy...,2023-12-14T08:18:50Z,Value locked on Cardanos ecosystem grew rapidl...


In [ ]:
from transformers import pipeline

In [ ]:
sent_pipeline = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:
sent_pipeline("the food was bland, the show was okay")

[{'label': 'POSITIVE', 'score': 0.9938115477561951}]